In [1]:
import matlab.engine
import numpy as np
import scipy.optimize

# Trial and pd parameters

In [2]:
# trial and adherence information
drug = "DTG"

days = np.double(40) #days to run trial for
increment = 0.01 # days between each timepoint returned
prob_adh = 0.5 if drug != 'TEST' else 0. # probability of taking each pill (independent)
adh_pat = 0 # 0 = random prob for each dose, 1 = random prob each day after missed dose
adh_shuff = np.double(0.) # standard deviation in the time of taking a pill vs scheduled (days)
trial_type = 1 # 1 = suppression trial, 0=maintenance trial
burnin = 7*5 # days before interval sampling begins in maintenance trials
sampling_int = 28 # days between each sampling time in maintenance trials
threshold = 200 # threshold viral load above which failure is declared
mut_frac = 0.2 # threshold fraction of population of mutant above which declared failure via resistance

## Use existing matlab scripts to calculate concentration(t)

In [3]:
eng = matlab.engine.start_matlab()
eng.cd(r'~/develop/withinhostHIV/MatlabCode/', nargout=0)
eng.eval("addpath('Parameters','Utilities')", nargout=0)

In [4]:
eng.workspace["drug"] = drug
trial, pd = eng.trial_setup(eng.eval("drug"), days, increment, prob_adh, adh_pat, adh_shuff, trial_type, burnin, sampling_int, threshold, mut_frac, nargout=2)
eng.workspace["trial"] = trial
eng.workspace["pd"] = pd

In [5]:
dose_t, _ = eng.adh_trajectory(eng.eval("pd.num_doses"), trial, nargout=2)

In [6]:
c_vec,inhib_vec = eng.drug_trajectory(pd, trial, dose_t, nargout=2)

## Use existing matlab scripts to calculate mutation matrix Q

In [7]:
smin=0.05 #highest mutant fitness
smax=0.9 #lowest mutant fitness
smiss=0.05 #fitness of strains with missing cost
rfmiss=1 #fold change in resistance for strains with it missing
mfmiss=0 #fractional change in slope for strains with it missing
back_mutation_on = drug != 'TEST' #include(1) or exclude(0) back mutation
direct_multi_hit = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from WT
direct_multi_multi = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from one res strain to another

In [8]:
mparams,Q = eng.getMutInfo(drug,smin,smax,smiss,rfmiss,mfmiss,back_mutation_on,direct_multi_hit,direct_multi_multi,nargout=2);
eng.workspace["Q"] = Q
Q = eng.eval("Q{:,:};") # get a matrix instead of a matlab table

## Viral dynamics parameters

In [9]:
R00 = 10.0 # average number of new virions produced by infection with a single virion === average number of new infected cells produced by 1 infected cell
fbp = 0.55 # fraction of blood that is plasma
Vpl = 3*10e3 # volume of plasma in mL
ftcell_pl = 0.02 # fraction of T cells that are circulating in blood (vs in lymph system)
hl_lr = 44*30.5 # half life of decay of latent reservoir, days
A = 100 # total reactivation of latent cells per day
flr = 1e-6# fraction of CD4 T cells that are latently infected at equilibrium

scale_cd4_body = (Vpl*10**3)/(fbp*ftcell_pl) # factor to go from T cell per ul blood to whole body

fa = .01 # fraction of infected CD4 cells that support productive vs abortive infection
dx = 0.05 # death rate of uninfected cells (per day) and by assumption dx == d, rate of death without viral cytolytic effects
L = scale_cd4_body*1000*dx # uninfected cells produced per day (/ul)
a = A/(flr*L/dx) # rate of exit of latent cells (per day)
dz = np.log(2)/(hl_lr)-a # death rate of latently infected cells (per day)

dy = 1 # TOTAL death rate of infected cells (per day) (=death due to burst (k) + death without viral cytolytic effects)
k = dy-dx # rate of death+emission of a burst of virions (per day)
p_est = 4.2e-4
N = scipy.optimize.least_squares(lambda N: R00*(1-(1-p_est)**N) - N*p_est, 100000, bounds=(10.0, np.inf))
assert(N.success)
N = N.x
assert(N > R00)

c = 23.04 # clearance of virions (per day) Ramratnam et al. Lancet 1999 [calculated as average over the 4 patients]
dv = c # APPROXIMATION
beta = R00 * dx * dy * dv / (L * (fa) * k * N)
# b = production of eclipsed cells in the v-steady model (which assumes \beta*X << c)
b = (k*N/c)*beta

# b *= fa

g = flr*dy/dx*(a+dz)/(fa*(1-1/R00)) # fraction of new infections that become latent

m = 1.0 # combined rate of maturation + death of eclipsed cells [right now not drawn from any source]

In [10]:
# Access matlab parameters
eng.workspace["mparams"] = mparams
IC50 = eng.eval('pd.IC50')
m = eng.eval('pd.m')
cost = eng.eval('mparams.cost')
rf = eng.eval('mparams.rf')
mf = eng.eval('mparams.mf')
msb = np.asarray(eng.eval("mparams.msb"))
t_vec = eng.eval('(0:trial.increment:trial.days);')


## Check beta(t) (from matlab) against beta(t) (from python)

In [ ]:
# REASSIGN SO THAT BETA -> B for calculations below
beta = b
beta

In [ ]:
IC50, m

In [13]:
_beta_t, _beta_u_t = eng.calculate_beta_of_t(t_vec, beta, c_vec, IC50, m, cost, rf, mf, eng.eval('height(mparams)'), nargout=2)

In [14]:
from numba import jit

@jit(nopython=True)
def calculate_beta_t(beta_0, concentration, IC50, m, cost, rf, mf):
    # B/[cost * (1 + concentration)/(IC50*rf)^(m*(1+mf))]
    B = beta_0
    denominator = cost * (1 + (concentration/(IC50*rf))**(m*(1+mf)))
    return B/denominator

# make sure we didn't mess up our function definition
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T), np.asarray(_beta_u_t)))
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1), np.asarray(_beta_t)))

beta_u_t = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T)
beta_t   = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1)

beta_t = np.concatenate([beta_t, beta_u_t], axis=1)
t_vec = np.asarray(t_vec).squeeze()
dose_t = np.asarray(dose_t).T.squeeze()

In [15]:
sampled_concentration = np.asarray(c_vec)

increment = eng.eval('trial.increment')

# closed around parameters
# pulls the nearest beta pre-calculated to the left of the current time
@jit(nopython=True)
def discrete_beta_of_t(t):
    # last time where beta was precalculated *before* t
    # specifically want *before* t so that we don't look ahead while integrating past a discontinuity
    t_discrete = int(np.floor(t/increment))
    return beta_t[t_discrete]

Q = np.asarray(Q)
@jit(nopython=True)
def force_of_infection(t):
    beta = discrete_beta_of_t(t)
    # calculate force of infection and flatten in row major order, which will make the matrix
    # read correctly in our 1-dimensional vector of rates
    return (np.expand_dims(beta,1) * Q).flatten()

@jit(nopython=True)
def diminished_force_of_infection(t):
    beta = discrete_beta_of_t(t) * (fa)
    # calculate force of infection and flatten in row major order, which will make the matrix
    # read correctly in our 1-dimensional vector of rates
    return (np.expand_dims(beta,1) * Q).flatten()

@jit(nopython=True)
def combined_productive_unproductive_force_of_infection(t):
    beta = discrete_beta_of_t(t)
    beta_productive = beta * fa
    beta_unproductive = beta * (1 - fa)

    # calculate force of infection including mutation and flatten in row major order,
    # which will make the matrix read correctly in our 1-dimensional vector of rates
    productive_ks = (np.expand_dims(beta_productive,1) * Q).flatten()
    # if we weren't in jit we would want:
    # np.concatenate([productive_ks, unproductive_ks])
    foi = np.zeros(len(productive_ks)+len(beta_unproductive))
    foi[:len(productive_ks)] = productive_ks
    # since unproductive infections don't produce new strains, we can ignore mutation
    foi[len(productive_ks):] = beta_unproductive
    return foi

In [ ]:
combined_productive_unproductive_force_of_infection(0.0)

In [ ]:
((np.expand_dims(discrete_beta_of_t(0),1) * Q)).flatten()

TODO: triple check to make sure this is the right orientation of beta_i relative to Q

# Define model

In [ ]:
import reactionmodel.model as reactionmodel
num_strains = beta_t.shape[1]
num_strains

In [ ]:
N*b*L/(dx*c) * (fa)

In [20]:
ys = [reactionmodel.Species(f'Y{i}', f'infected cells strain {i}') for i in range(num_strains)]
zs = [reactionmodel.Species(f'Z{i}', f'latent cells strain {i}', ) for i in range(num_strains)]
ws = [reactionmodel.Species(f'W{i}', f'eclipsed cells strain {i}') for i in range(num_strains)]
x = reactionmodel.Species("X", "target cells")
all_species = [x] + ys + ws + zs

A steady-state approximation for virus copies:

$$ v = a \cdot \text{burst size} \cdot y / u $$

$$ \text{burst size} = kN / a $$

In [21]:
target_cell_birth = reactionmodel.Reaction([], [x], description="target cell birth", k=L)
target_cell_death = reactionmodel.Reaction([x], [], description="target cell death", k=dx)

infected_death = [reactionmodel.Reaction([ys[i]], [], description=f"death of infected cell strain {i}", k=(dy)) for i in range(num_strains)]
into_latency = [reactionmodel.Reaction([ys[i]], [zs[i]], description=f"strain {i} --> latency", k=g) for i in range(num_strains)]
out_of_latency = [reactionmodel.Reaction([zs[i]], [ys[i]], description=f"latent --> strain {i}", k=a) for i in range(num_strains)]
death_of_latent = [reactionmodel.Reaction([zs[i]], [], description=f"latent {i} death", k=dz) for i in range(num_strains)]

# INCLUDE FACTOR OF FA BELOW IF MODELING UNPRODUCTIVELY INFECTED ECLIPSED CELLS
maturation_of_eclipse = [reactionmodel.Reaction([ws[i]], [ys[i]], description=f"productive maturation of strain {i} eclipsed", k=m) for i in range(num_strains)]

#eclipse_death = [reactionmodel.Reaction([ws[i]], [], description=f"death of strain {i} eclipsed", k=m*(1-fa)) for i in range(num_strains)]


In [22]:
productive_infections = []
unproductive_infections = []
for i in range(num_strains):
    unproductive_infections.append(reactionmodel.Reaction([x, ys[i]], [ys[i]], description=f"unproductive infection of x by {i}->empty set"))
    for j in range(num_strains):
        productive_infections.append(reactionmodel.Reaction([x, ys[i]], [ys[i], ws[j]], description=f"productive infection of x by {i}->{j}"))

infection_family = reactionmodel.ReactionRateFamily(productive_infections + unproductive_infections, k=combined_productive_unproductive_force_of_infection)

In [23]:
infections = []
for i in range(num_strains):
    for j in range(num_strains):
        infections.append(reactionmodel.Reaction([x, ys[i]], [ys[i], ws[j]], description=f"infection of x by {i}->{j}"))

# UNCOMMENT TO REVERT MODEL CHANGES
#infection_family = reactionmodel.ReactionRateFamily(infections, k=diminished_force_of_infection)


all_reactions = [target_cell_birth, target_cell_death] + infected_death + into_latency + out_of_latency + death_of_latent + maturation_of_eclipse + [infection_family]

In [ ]:
model = reactionmodel.Model(all_species, all_reactions)
len(model.all_reactions)

# Run forward simulation

In [25]:
from hybrid.hybrid import HybridSimulator, FixedThresholdPartitioner, NThresholdPartitioner

In [ ]:
ys

In [27]:
extra_discontinuities = list(np.linspace(0.0,40.0,400))
extra_discontinuities = []

y0 = np.zeros(len(all_species))
y0[model.species_index[x]] = (L/dx/R00)

expanded_msb = np.squeeze(np.vstack([np.array([1]), msb]))
y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] = fa*(1-1/R00)*L/dy * expanded_msb
y0[model.species_index[zs[0]] : model.species_index[zs[-1]] + 1] = y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] * g/(a+dz)
y0[model.species_index[ws[0]] : model.species_index[ws[-1]] + 1] = y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] * y0[model.species_index[x]] * b * fa / (m)

In [ ]:
y0[model.species_index[x]]

In [ ]:
fa*(1-1/R00)*L/dy

## N specimen partitioning

In [30]:
# the adherence pattern and hence the R0s is being determined in matlab before any of this code runs,
# so don't be scared if trajectories look almost identical with different seeds in python

simulator = HybridSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    NThresholdPartitioner(100),
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    fast_scale='langevin',
    approximate_rtot=True,
    contrived_no_reaction_rate=20.0,
    jit=True,
    species_labels=[s.name for s in model.species],
    pathway_labels=[r.description for r in model.all_reactions]
)

In [ ]:
rng = np.random.default_rng(0)
result = simulator.simulate([0, 40.0], y0, rng)

In [ ]:
result.status_counter

In [ ]:
result.pathway_counter

In [ ]:
labs = [r.description for r in model.all_reactions]

labs[352]

In [ ]:
import matplotlib.pyplot as plt

mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[0].T, label='x')
plt.yscale('log')
plt.ylabel('total uninfected CD4 cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e14])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('total infected cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e13])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species_index[ws[0]]:model.species_index[ws[-1]]+1].T, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('total eclipse cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e12])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(ys[0]):model.species.index(ys[-1])+1].T*k*N/c/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])

## Tau leaping

In [36]:
from hybrid.tau import TauLeapSimulator

t_simulator = TauLeapSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    discontinuities=dose_t,
    jit=True,
    species_labels=[s.name for s in model.species],
    pathway_labels=[r.description for r in model.all_reactions],
    epsilon=0.01,
    time_handling='inhomogeneous_monotonic_homogeneous_gillespie',
    #species_creation_is_critical=True,
)

In [ ]:
t_simulator = TauLeapSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    discontinuities=dose_t,
    jit=True,
    species_labels=[s.name for s in model.species],
    pathway_labels=[r.description for r in model.all_reactions],
    epsilon=0.01,
    time_handling='homogeneous',
    method='implicit'
    #species_creation_is_critical=True,
)

In [ ]:
rng = np.random.default_rng(0)
result = t_simulator.simulate([0.0, 40.0], y0, rng)

In [ ]:
result.status_counter

In [31]:
%load_ext jupyterflame

In [ ]:
%%flame
rng = np.random.default_rng(0)
t_simulator.simulate([0, 40.0], y0, rng)

In [ ]:
result.status_counter

In [ ]:
taus = [result.t_history[i+1] - result.t_history[i] for i in range(len(result.t_history)-1)]
plt.hist(taus)

In [ ]:
import matplotlib.pyplot as plt

mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[0].T, label='x')
plt.yscale('log')
plt.ylabel('total uninfected CD4 cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e14])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('total infected cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e9])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species_index[ws[0]]:model.species_index[ws[-1]]+1].T, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('total eclipse cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e12])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(ys[0]):model.species.index(ys[-1])+1].T*k*N/c/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])